In [25]:
# Packages
import pandas as pd
import numpy as np

## Brief description of the dataset and a summary of its attributes
We import the dataset and explore its characteristics. The dataset contains one numerical variable and 8 objects, that are either strings or dates. The measurements are monthly measurements and the date column, holds the respective dates.

In [26]:
df = pd.read_csv('data.csv')

In [27]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50799 entries, 0 to 50798
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      50799 non-null  int64  
 1   date            50799 non-null  object 
 2   datatype        50799 non-null  object 
 3   station         50799 non-null  object 
 4   attributes      50778 non-null  object 
 5   value           50799 non-null  float64
 6   countryid       50799 non-null  object 
 7   id              50799 non-null  object 
 8   datacategoryid  50799 non-null  object 
 9   name            50799 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.9+ MB
None


The dataset includes 10 different features on temperature and precipitation.

In [28]:
print(df[['datacategoryid', 'id', 'name']].drop_duplicates(ignore_index=True))

   datacategoryid    id                                               name
0            TEMP  CDSD                 Cooling Degree Days Season to Date
1            TEMP  EMNT        Extreme minimum temperature for the period.
2            TEMP  EMXT        Extreme maximum temperature for the period.
3            TEMP  HDSD                 Heating Degree Days Season to Date
4            TEMP  TAVG                               Average Temperature.
5            TEMP  TMAX                                Maximum temperature
6            TEMP  TMIN                                Minimum temperature
7            PRCP  DSND  Number days with snow depth > 1 inch(25.4mm) f...
8            PRCP  EMSD         Extreme maximum snow depth for the period.
9            PRCP  EMXP      Extreme maximum precipitation for the period.
10           PRCP  PRCP                                      Precipitation


In [29]:
print(min(df['date']))
print(max(df['date']))

2014-01-01T00:00:00
2023-06-01T00:00:00


The measurements cover the period of Jan. 2014 - June 2023.

## Initial plan for data exploration
We cast each feature to a numerical variable and we check the basic statistics of those variables.

In [30]:
df.pivot(index = ['date', 'station'], columns = 'id', values = 'value')

id                                      CDSD  DSND  EMNT  EMSD  EMXP  EMXT  \
date                station                                                  
2014-01-01T00:00:00 GHCND:FG000081405    NaN   NaN   NaN   NaN  85.1   NaN   
                    GHCND:FGM00081401    NaN   NaN  19.7   NaN  35.1   NaN   
                    GHCND:FGM00081415    NaN   NaN  20.1   NaN  25.9   NaN   
                    GHCND:FP000091925    NaN   NaN   NaN   NaN  30.0  32.7   
                    GHCND:FP000091948    NaN   NaN   NaN   NaN  29.0   NaN   
...                                      ...   ...   ...   ...   ...   ...   
2023-06-01T00:00:00 GHCND:GRM00016622    NaN   NaN   NaN   NaN  37.3   NaN   
                    GHCND:GRM00016719  185.5   NaN  13.0   NaN   0.5  33.8   
                    GHCND:GRM00016726    NaN   NaN   NaN   NaN   1.8  33.4   
                    GHCND:MFM00067005    NaN   NaN   NaN   NaN   0.3   NaN   
                    GHCND:SBM00071805    NaN   NaN   NaN   NaN  17.8   NaN   

id                                      HDSD   PRCP   TAVG   TMAX   TMIN  
date                station                                               
2014-01-01T00:00:00 GHCND:FG000081405    NaN  594.7    NaN    NaN    NaN  
                    GHCND:FGM00081401    NaN  233.7    NaN    NaN  22.02  
                    GHCND:FGM00081415    NaN  161.8    NaN    NaN  22.32  
                    GHCND:FP000091925    NaN   81.4    NaN  30.73    NaN  
                    GHCND:FP000091948    NaN  126.4    NaN    NaN    NaN  
...                                      ...    ...    ...    ...    ...  
2023-06-01T00:00:00 GHCND:GRM00016622    NaN   94.3    NaN    NaN    NaN  
                    GHCND:GRM00016719  938.2    0.5  23.59  29.34  17.83  
                    GHCND:GRM00016726    NaN    3.9    NaN  29.57    NaN  
                    GHCND:MFM00067005    NaN    2.4    NaN    NaN    NaN  
                    GHCND:SBM00071805    NaN   76.8    NaN    NaN    NaN  

[9777 rows x 11 columns]

In [31]:
df_summary = df.groupby(['countryid', 'name', 'id'])['value'].agg([np.mean, np.median, np.std, min, max, lambda x : np.quantile(x, 0.25), 
                                                                   lambda x : np.quantile(x, 0.75), lambda x: x.count()])\
    .rename(columns = {"<lambda_0>":"25 percentile", "<lambda_1>":"75 percentile", "<lambda_2>":"Count"})

print(df_summary)

                                                                         mean  \
countryid name                                               id                 
FIPS:FR   Average Temperature.                               TAVG   13.641054   
          Cooling Degree Days Season to Date                 CDSD  147.630388   
          Extreme maximum precipitation for the period.      EMXP   20.382597   
          Extreme maximum snow depth for the period.         EMSD  214.636364   
          Extreme maximum temperature for the period.        EMXT   24.945502   
          Extreme minimum temperature for the period.        EMNT    3.745144   
          Heating Degree Days Season to Date                 HDSD  962.318617   
          Maximum temperature                                TMAX   18.299411   
          Minimum temperature                                TMIN    9.418633   
          Number days with snow depth > 1 inch(25.4mm) fo... DSND   20.454545   
          Precipitation     

## Data cleaning and feature engineering
we explore the dataset in its initial form, looking for outliers. We also simplify by removing the stations feature. Finally we explore the distribution of variables and look for the need of transformations or scaling. We do not need to deal with categorical variables. We only have numerical variables.

We get the interquartile range, and look for outliers.

In [32]:
df_summary['iqr'] = df_summary['75 percentile'] - df_summary['25 percentile']

In [33]:
df_summary['upper_limit'] = df_summary['75 percentile'] + 1.5 * df_summary['iqr']
df_summary['lower_limit'] = df_summary['25 percentile'] - 1.5 * df_summary['iqr']

In [34]:
display(df_summary[['upper_limit', 'lower_limit']])

upper_limit  \
countryid name                                               id                  
FIPS:FR   Average Temperature.                               TAVG     32.69875   
          Cooling Degree Days Season to Date                 CDSD    605.00000   
          Extreme maximum precipitation for the period.      EMXP     46.15000   
          Extreme maximum snow depth for the period.         EMSD    410.00000   
          Extreme maximum temperature for the period.        EMXT     50.05000   
          Extreme minimum temperature for the period.        EMNT     22.70000   
          Heating Degree Days Season to Date                 HDSD   4182.25000   
          Maximum temperature                                TMAX     40.42125   
          Minimum temperature                                TMIN     26.87000   
          Number days with snow depth > 1 inch(25.4mm) fo... DSND     43.00000   
          Precipitation                                      PRCP    171.60000   
FIPS:GR   Average Temperature.                               TAVG     40.87250   
          Cooling Degree Days Season to Date                 CDSD   1743.75000   
          Extreme maximum precipitation for the period.      EMXP     62.35000   
          Extreme maximum temperature for the period.        EMXT     53.68750   
          Extreme minimum temperature for the period.        EMNT     31.85000   
          Heating Degree Days Season to Date                 HDSD    155.06250   
          Maximum temperature                                TMAX     47.21875   
          Minimum temperature                                TMIN     35.03000   
          Precipitation                                      PRCP    155.75000   

                                                                   lower_limit  
countryid name                                               id                 
FIPS:FR   Average Temperature.                               TAVG     -5.35125  
          Cooling Degree Days Season to Date                 CDSD   -363.00000  
          Extreme maximum precipitation for the period.      EMXP    -11.85000  
          Extreme maximum snow depth for the period.         EMSD     -6.00000  
          Extreme maximum temperature for the period.        EMXT      0.05000  
          Extreme minimum temperature for the period.        EMNT    -15.70000  
          Heating Degree Days Season to Date                 HDSD  -2384.55000  
          Maximum temperature                                TMAX     -3.70875  
          Minimum temperature                                TMIN     -8.37000  
          Number days with snow depth > 1 inch(25.4mm) fo... DSND     -1.00000  
          Precipitation                                      PRCP    -54.80000  
FIPS:GR   Average Temperature.                               TAVG      0.17250  
          Cooling Degree Days Season to Date                 CDSD  -1046.25000  
          Extreme maximum precipitation for the period.      EMXP    -30.05000  
          Extreme maximum temperature for the period.        EMXT      2.78750  
          Extreme minimum temperature for the period.        EMNT     -8.15000  
          Heating Degree Days Season to Date                 HDSD    -93.03750  
          Maximum temperature                                TMAX      0.26875  
          Minimum temperature                                TMIN     -2.85000  
          Precipitation                                      PRCP    -82.05000

In [ ]:
# Just some cleaning of the main table
df.reset_index()
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [52]:
df = df.merge(df_summary[['upper_limit', 'lower_limit']], on = ['countryid', 'id'], how = "left")

In [58]:
# Check for outliers
outliers = df[(df['value']>df['upper_limit'])|(df['value']<df['lower_limit'])]

In [59]:
outliers.shape

(1296, 11)

We find that approx. 1000 rows in 50000 rows of dataset are extreme values, which is around 2%. Since there are several extreme values, we are going to keep them.

In [38]:
df.index

RangeIndex(start=0, stop=50799, step=1)

In [53]:
display(df)

,date,datatype,station,attributes,value,countryid,id,datacategoryid,name,upper_limit,lower_limit
0,2014-01-01T00:00:00,CDSD,GHCND:FR000007130,E,0.0,FIPS:FR,CDSD,TEMP,Cooling Degree Days Season to Date,605.00,-363.00
1,2014-01-01T00:00:00,CDSD,GHCND:FR000007190,E,0.0,FIPS:FR,CDSD,TEMP,Cooling Degree Days Season to Date,605.00,-363.00
2,2014-01-01T00:00:00,CDSD,GHCND:FR000007255,E,0.0,FIPS:FR,CDSD,TEMP,Cooling Degree Days Season to Date,605.00,-363.00
3,2014-01-01T00:00:00,CDSD,GHCND:FR000007510,E,0.0,FIPS:FR,CDSD,TEMP,Cooling Degree Days Season to Date,605.00,-363.00
4,2014-01-01T00:00:00,CDSD,GHCND:FR000007630,E,0.0,FIPS:FR,CDSD,TEMP,Cooling Degree Days Season to Date,605.00,-363.00
...,...,...,...,...,...,...,...,...,...,...,...
50794,2023-06-01T00:00:00,PRCP,GHCND:GR000016754,",,,S",43.2,FIPS:GR,PRCP,PRCP,Precipitation,155.75,-82.05
50795,2023-06-01T00:00:00,PRCP,GHCND:GR000167230,"1,,,S",0.0,FIPS:GR,PRCP,PRCP,Precipitation,155.75,-82.05
50796,2023-06-01T00:00:00,PRCP,GHCND:GRM00016622,",,,S",94.3,FIPS:GR,PRCP,PRCP,Precipitation,155.75,-82.05
50797,2023-06-01T00:00:00,PRCP,GHCND:GRM00016719,",,,S",0.5,FIPS:GR,PRCP,PRCP,Precipitation,155.75,-82.05


For each day, we can have measurements taken at several stations. We can summarize those measurements and reduce the size of the dataset.

In [74]:
df_mean = df.groupby(['countryid', 'datacategoryid', 'datatype', 'name', 'date'])[['value']].agg(["mean", "median"])

In [86]:
display(df_mean)

value  \
                                                                                                               mean   
countryid datacategoryid datatype name                                               date                             
FIPS:FR   PRCP           DSND     Number days with snow depth > 1 inch(25.4mm) fo... 2014-02-01T00:00:00  28.000000   
                                                                                     2014-03-01T00:00:00  19.000000   
                                                                                     2015-01-01T00:00:00  12.000000   
                                                                                     2015-02-01T00:00:00  14.000000   
                                                                                     2015-03-01T00:00:00  19.000000   
...                                                                                                             ...   
FIPS:GR   TEMP           TMIN     Minimum temperature                                2023-02-01T00:00:00   5.080000   
                                                                                     2023-03-01T00:00:00   8.010000   
                                                                                     2023-04-01T00:00:00  10.546667   
                                                                                     2023-05-01T00:00:00  13.850000   
                                                                                     2023-06-01T00:00:00  19.026667   

                                                                                                                 
                                                                                                         median  
countryid datacategoryid datatype name                                               date                        
FIPS:FR   PRCP           DSND     Number days with snow depth > 1 inch(25.4mm) fo... 2014-02-01T00:00:00  28.00  
                                                                                     2014-03-01T00:00:00  19.00  
                                                                                     2015-01-01T00:00:00  12.00  
                                                                                     2015-02-01T00:00:00  14.00  
                                                                                     2015-03-01T00:00:00  19.00  
...                                                                                                         ...  
FIPS:GR   TEMP           TMIN     Minimum temperature                                2023-02-01T00:00:00   5.08  
                                                                                     2023-03-01T00:00:00   8.01  
                                                                                     2023-04-01T00:00:00   9.98  
                                                                                     2023-05-01T00:00:00  13.85  
                                                                                     2023-06-01T00:00:00  18.40  

[1804 rows x 2 columns]

In [101]:
df_mean.columns = ['_'.join(col) for col in df_mean.columns.values]

In [102]:
df_mean.columns.values

array(['value_mean', 'value_median'], dtype=object)